# Particle Filtering: How and Why it works

### Use cases for Particle Filter

1. Multi-modal problems: Tracking multiple vehicles on a highway using radar data. Each vehicle represents a different mode in the state distribution, and the particle filter can handle the multiple hypotheses effectively.

2. Occluded Objects : In computer vision, a pedestrian walking behind a parked car becomes temporarily hidden from a camera's view. A particle filter can predict the pedestrian's position during occlusion by maintaining multiple possible states.

3. Systems with non-linear behavior : An aircraft navigating through turbulent weather where wind forces cause non-linear effects on its trajectory. Another example is a robot arm with joint friction and backlash, leading to non-linear motion dynamics.

4. Systems providing non-linear measurements : A robot using a laser rangefinder measures distances to obstacles. Converting these measurements to Cartesian coordinates involves trigonometric functions, introducing non-linearity.

5. System with non-gaussian noise : A sensor affected by electromagnetic interference, causing occasional large errors (outliers) in measurements. The noise distribution might be heavy-tailed (e.g., a Cauchy distribution) rather than Gaussian.

6. System variables changing continiously : the object's position and velocity can smoothly change over time.

7. multivariate problem: we want to track several attributes, such as position, velocity, turn rates, etc.

8. Unknown system model : we may not know the process model of the system.

#### None of the following filters will work well with all of the above constraints:

1. Discrete Bayes Fitler : Suitable for discrete state spaces but not efficient for continuous or high-dimensional problems.

2. Kalman Filter : Assumes linear system models and Gaussian noise, which limits its applicability to non-linear or non-Gaussian systems.

3. Unscented kalman Filter: Linearizes non-linear models around the current estimate, which can introduce significant errors if the system is highly non-linear.

4. Extended Kalman Filter : Handles non-linearities better than EKF but still assumes Gaussian noise and may not perform well with multi-modal distributions or non-Gaussian noise.



### What is it ? Tell me in Simple words

In very simple words, Let us take a robot localization problem you will take a finite number of particles and sprinkle them in the environment around your robot which you want to track. Then using prediction, update and resampling steps you get a nice estimate of your robots location.

*side note: Sampling simply means generating a set of random possible states (x,y,z, oritentation) of the robot from a probability distribution. It can be any sort of distribution !*

There are three main steps that you need to keep in mind: 

1. Predict 
2. Update 
3. Resample

And particle filter algorithm will just run an iterative prediction-correction-Resample loop. The good thing is that it can estimate the states despite the process and measurement noise, which can be of any arbitrary non-gaussain distribution. Process noise is simply the uncertainty in how the system's state changes over time due to unpredictable influences or model inaccuracies while Measurement noise is uncertainty in the observations caused by sensor errors/noise.

Below you can view how a generic particle filter algorithm is structured

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display, Image

img = plt.imread("flowchart.png")
plt.figure(figsize=(10, 10))
plt.imshow(img)
plt.title("Flow chart representing a generic Particle filter algorithm")
plt.axis('off')
plt.show()

Let's discuss the basic particle filter algorithm for the robot localization problem. We have a robot whose orientation and velocity can be controlled by the user. It also has sensors that measure distances to obstacles. Both the sensors and the control mechanisms have noise, and our goal is to track the robot's position despite this uncertainty.

##### Before going any further, let's clarify what particles are:

* Particles: A particle is a simulated possible position and orientation of the robot. Each particle represents a belief about where the robot could be located. So, each particle has a position (x, y) and an orientation angle. Additionally, particles have an associated weight. You can think of all of the particles representing a probability distribution, locations where there are more particles represent a higher belief(that robot is at the position), and locations with fewer particles represents a lower belief(robot is likely not there). If there was a large clump of particles near a specific location that would imply that we are more certain that the robot is there.

* Weight: The weight indicates how likely it is that the particle represents the robot's true state. A higher weight means the particle more closely matches the robot's actual position.

##### Now, let's go through the steps of the particle filter algorithm:

*1. Randomly Generate Particles:* We start by randomly placing particles around an initial guessed position of the robot. If no guess is available, we can spread the particles uniformly across the entire map. Each particle is initialized with the same weight and has a certain (x,y) position and orientation.

*2. Predict the Next State of the Particles:* As the robot moves, we update all the particles based on the robot's control inputs and a motion model that includes some randomness to account for movement noise. For example, if the robot moves 1 meter forward and rotates 20 degrees, we move each particle accordingly but also add slight variations to simulate the uncertainty in motion.

*3. Update Weights:* We update the weights of all particles by comparing their predicted sensor readings to the actual sensor data received (e.g., from a 2D lidar). Particles whose predicted measurements closely match the actual sensor data receive higher weights, indicating they are more likely to represent the robot's true state.

*4. Resample:* We remove particles with low weights and replace them with new particles sampled from those with higher weights. This step concentrates the particles around the more probable (higher weight) states.

*5. Compute Estimate:* To estimate the robot's current position, we compute the weighted average of the particles' positions and orientations.

We repeat these steps multiple times. As the algorithm iterates, the particles converge around the robot's true position. This allows us to accurately track the robot's movement despite the noise in sensors and control inputs. Observe the gif below :) 

In [ ]:
from IPython.display import display, HTML
display(HTML('<a href="https://drive.google.com/file/d/18LyPVCw7A8dPH7s46XfZRgiz9-doLP_u/view?usp=sharing" target="_blank">Figure 2: Animation of generic particle filter algorithm</a>'))

Let us go over the important steps in bit of a detail:

### Predict Step Exaplained 

In the predict step of the particle filter, each particle represents a possible position and orientation of the robot. Lets think about it, when we send a command to the robot—for example, to move forward by 0.1 meters and turn by 0.007 radians— we can also move all the particles exactly these amounts (wherever they might be). However, this would cause a problem because the robot's movements are not perfect (remember noise in control system of robot); it won't move exactly as commanded due to control noise and uncertainties.

To address this, we add randomness (noise) to the movement of each particle. Instead of moving each particle exactly by the commanded amounts, we add some noise to the command and move the particle with that modified command. This means each particle moves slightly differently, simulating the imperfections in the robot's actual movement.

    modified command = noise + user command 

By modeling the uncertainty in the robot's motion, the particle filter accurately captures the range of possible positions the robot could be in. This ensures that the filter correctly represents our belief about the robot's position as a probability distribution, accounting for the inherent noise in the control system.


In [ ]:
image = mpimg.imread('predict_step_dispersion.png')
plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.title("Figure 3: During the predict step, particles are propagated forward by adding\nsome noise to user command according to odometry motion model.")
plt.axis('off')
plt.show()

### Update Step Explained 

Before diving into the update step of the particle filter algorithm, it's important to understand the concept of likelihood. Likelihood is a key idea in Bayes' theorem, and since the particle filter is based on Bayesian principles, grasping this concept is essential. 

Think of likelihood as a score that indicates how likely it is for something to happen, given some evidence.

#### So taking the robot localization problem, 

##### *Particles as Hypotheses:* 

Each particle represents a possible position and orientation of the robot—a hypothesis about where the robot might be.

##### *Calculating Likelihood:*

We compare the expected 2D sensor readings if the sensor was located at each of the particle's position to the actual sensor readings the robot receives. If a particle's expected readings closely match the actual readings, the likelihood for that particle is high. 

Example: Assume a particle A at x=1,y=1. Robot with sensor is at x=0, y=0. We would compute the sensor reading if the sensor was at the particle's position (x=1, y=1) and then compare by taking a difference of that sensor reading with the actual sensor reading at x=0,y=0 which is the true sensor reading. We do this for all the particles and proportional to that difference we assign a weight. 

##### *Updating Weights:*

The likelihood determines the weight of each particle. Particles with higher likelihoods (better matches to the observed/true data) receive higher weights.

In short these are the sub-steps in this Update Step: 
1. Process incoming sensor measurements
2. Compare them with simulated measurments of the particles on the map
3. Prioritize particles where measurements and map have the lowest error

##### Follow the images below:

Step 1: Robot observes the environment by acquiring distances from the range sensor position toward nearby obstacles.

In [ ]:
image = mpimg.imread('laser_scans_real_robot.png')
plt.imshow(image)
plt.title('Starting Robot position with the area captured by laser scan')
plt.axis('off')
plt.show()

Step 2: The filter simulates for each particle a laser scan by raycasting against the internal map at each particle's pose.

In [ ]:
image1 = mpimg.imread('p1.png')
image2 = mpimg.imread('p2.png')
image3 = mpimg.imread('p3.png')
image4 = mpimg.imread('p4.png')
fig, axes = plt.subplots(1, 4, figsize=(12, 3))
axes[0].imshow(image1)
axes[0].axis('off')
axes[1].imshow(image2)
axes[1].axis('off')
axes[2].imshow(image3)
axes[2].axis('off')
axes[3].imshow(image4)
axes[3].axis('off')
fig.suptitle('Raytracing for each particle: the region of scan area it captures', fontsize=14)
plt.subplots_adjust(top=0.85)
plt.show()



Step 3: The degree of matching between distances obtained by the real laser scan and the distances calculated by each particle's expected distance measurement determines the likelihood of a particle being in the vicinity of the robot's ground truth pose. During this process, a new weight is assigned
to each particle corresponding to how close the each particles predicted scan is to the real robot laser scan

In [ ]:
image = mpimg.imread('scan_matching.png')
plt.figure(figsize=(10, 6))
plt.imshow(image)
plt.title('Scan Matching')
plt.axis('off')
plt.show()


Particles with the highest weight are used to estimate the position of the robot. Particles with small weights are discarded.

Notice how the amount of unique structural features in the environment is important: the more, the better. If the environment is ambiguous, for example, with large empty walls, with no edges, with no windows, no doors, or other features that can provide relevant information for the robot to determine its pose, the particle filter will not perform well.

### Compute the State Estimate

To estimate the robot's position after each update step we can compute the mean of the estimate as the sum of the weighted values of the particles. 

$$\displaystyle \mu = \frac{1}{N}\sum_{i=1}^N w^ix^i$$

Let's give an example:

Suppose we have three particles:

- **Particle 1**:
  - Position: \( x^1 = (2, 3) \)
  - Weight: \( w^1 = 0.2 \)

- **Particle 2**:
  - Position: \( x^2 = (4, 5) \)
  - Weight: \( w^2 = 0.5 \)

- **Particle 3**:
  - Position: \( x^3 = (3, 4) \)
  - Weight: \( w^3 = 0.3 \)

Compute the estimated position:

For the x-coordinate:

$$
\mu_x = w^1 x^1_x + w^2 x^2_x + w^3 x^3_x \\
\mu_x = (0.2)(2) + (0.5)(4) + (0.3)(3) = 0.4 + 2.0 + 0.9 = 3.3
$$

For the y-coordinate:

$$
\mu_y = w^1 x^1_y + w^2 x^2_y + w^3 x^3_y \\
\mu_y = (0.2)(3) + (0.5)(5) + (0.3)(4) = 0.6 + 2.5 + 1.2 = 4.3
$$

So, the estimated position of the robot is:

$$
\mu = (\mu_x, \mu_y) = (3.3, 4.3)
$$


### Particle Resampling 



After the update step, the particles now have updated weights based on how well they match the robot's actual state. The next important step is resampling. Resampling is the process of replacing particles with low weights with new particles that are located in areas with higher probability (closer to where the robot likely is). 

#### Why do we need resampling ? 

Without resampling, particles would remain scattered across the map and wouldn't effectively use the new information from the sensor measurements. So, resampling is crucial because it helps the particles focus around the robot's true position. (Remember scan matching to remove the particles that are not well alligned with the scan )

#### Here's how resampling works:

*Selection of Particles:* We select a certain number of particles to carry over to the next iteration of Particle Filter Algorithm. Particles with higher weights are more likely to be chosen, meaning they're more likely to "survive." Particles with low weights are less likely to be selected and are effectively discarded.

*Generating New Particles:* We create new particles near the surviving high-weight particles. This means that more particles are placed in areas where the robot is more likely to be.

This process results in particles clustering into groups or "clouds." Each cloud represents a possible estimate of the robot's position. Because there may be several clusters, the filter is effectively considering multiple possible locations for the robot. This ability to represent multiple hypotheses is known as a multimodal distribution in statistics (as opposed to a single hypothesis, or unimodal distribution).

In [ ]:
image = mpimg.imread('resampling_particle_clouds.png')
plt.figure(figsize=(10, 6))
plt.imshow(image)
plt.title('Clusters of particles formed after each Resampling step')
plt.axis('off')
plt.show()

Eventually, the cluster with the highest combined weight will cover the area where the robot actually is. We use the average position (mean) and the spread (covariance) of that cluster to estimate the robot's position ( Just as we saw in Compute State Estimate Section)

#### What is AMCL ?
It is simply an extension of the standard particle filter known as the Adaptive Monte Carlo Localization (AMCL) algorithm. AMCL improves the resampling step by adjusting the number of particles dynamically:

*When Uncertainty is High:* If the robot's position is very uncertain (many particles have low weights), AMCL increases the number of particles. This allows the algorithm to explore more possible positions.

*When Confidence is High:* If the robot is confident about its position (particles have high weights), AMCL reduces the number of particles. This saves computational resources while maintaining accuracy.

AMCL uses a method called KLD-sampling to decide how to adjust the number of particles based on the desired level of accuracy.

#### What is Neff ? 
We donot have to perform resampling at every epoch that would be very computationally expensive. For instance, if you received no new sensor measurements than it doesnot make sensor to resample. So we can determine when to resample by using something called the effective N, which approximately measures the number of particles which meaningfully contribute to the probability distribution. If Neff falls below a certain threshold it is time to resample. 

Resampling is necessary to add diverstiy in the particles in our state space, we need diversity ! if Neff is low it means all the high weight is centered around a small set of particles. So we need to choose the threshold appropriately it could be Neff < N/2 which is the most common approach. 


### Important Parameters for Tuning the AMCL Node in ROS2 